In [13]:
from bs4 import BeautifulSoup
import os
import requests
import pandas as pd

In [74]:
url = "https://www.pagat.com/alpha/"
page = requests.get(url)

In [75]:
page

<Response [200]>

In [76]:
soup = BeautifulSoup(page.text, 'lxml')

In [77]:
table = soup.find_all('table')[0]

In [84]:
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 4:
        if "French suited cards" in str(cols[2].find('span')):
            data.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip(), cols[3].text.strip(), cols[0].find('a')['href']))

In [79]:
table.find_all('tr')[2].find_all('td')[2].find('span')['title']

'French suited cards'

In [80]:
table.find_all('tr')[2].find_all('td')[0].find('a')['href']

'../poker/variants/buyyourcard.html#abyssinia'

In [100]:
df = pd.DataFrame(data, columns=['Game', 'Players', 'Design', 'Quantity', 'href'])

In [101]:
df['Game'] = df['Game'].str.split('\n').str[0]

In [102]:
df['href'] = 'https://www.pagat.com' + df['href'].str[2:]

In [104]:
df.head()

,Game,Players,Design,Quantity,href
0,Abyssinia,3â10,,52,https://www.pagat.com/poker/variants/buyyourca...
1,Ace to Ace,2â6,,"52, 2x52",https://www.pagat.com/rummy/crazy.html
2,Ace-To-Five Lowball,"2, 3, 4â6, 7, 8",,"52, 52+J",https://www.pagat.com/poker/variants/lowball.h...
3,Ace-To-Six Lowball (Draw),"2, 3, 4â6, 7, 8",,52,https://www.pagat.com/poker/variants/lowball.h...
4,"Aces, Straights and Flushes","2â4, 5â8",,52,https://www.pagat.com/poker/variants/7stud.htm...


In [105]:
url2 = "https://www.pagat.com/rummy/5000rummy.html"
page2 = requests.get(url2)

In [107]:
soup2 = BeautifulSoup(page2.text, 'lxml')

In [146]:
group_content = {}
target = ''
go = 0
for tag in soup2.find_all('div', {'class': 'mainContent'})[0].find_all():
    if tag.name == 'h2':
        go = 1
        target = tag.text
        group_content[target] = ''
    else:
        if go == 1:
            group_content[target] += (' ' + tag.text)

In [145]:
soup2.find_all('div', {'class': 'mainContent'})[0].find_all()[21]

<p>This North American Rummy game exists in many  versions and has many different names. The play is similar to 500 Rummy in that a player may take several cards from the discard pile, but there two major differences.</p>

In [162]:
df2 = pd.DataFrame(group_content, index = ['Text']).T.reset_index()

In [165]:
all_text = pd.DataFrame()
for ref in df['href']:
    page2 = requests.get(ref)
    soup2 = BeautifulSoup(page2.text, 'lxml')
    group_content = {}
    target = ''
    go = 0
    for tag in soup2.find_all('div', {'class': 'mainContent'})[0].find_all():
        if tag.name == 'h2':
            go = 1
            target = tag.text
            group_content[target] = ''
        else:
            if go == 1:
                group_content[target] += (' ' + tag.text)
    all_text = all_text.append(pd.DataFrame(group_content, index = ['Text']).T.reset_index())

In [171]:
all_text.to_csv('text_data_grouped_by_cat.csv')

In [173]:
all_text['chars'] = all_text['Text'].str.len

,index,Text
0,Introduction,This is a collection of stud poker variants i...
1,Buy Your Card / Substitution,In this variant players can pay to exchange c...
2,Abyssinia,In this five card stud high-low variant playe...
3,The Price is Right,This is a seven-card stud high-low game for t...
4,Auction,This is another seven-card high-low game. Aft...
5,Grocery Store Dots,"This is a seven-card stud game. ""Dots"" are th..."
6,Wall Street,This seven card stud game is similar to The P...
0,Introduction,This Rummy game consists of series of deals...
1,Players and Cards,"From 3 to 6 people can play, using a standard..."
2,Objective and Melds,A complete game consists of 13 deals. In each...
